# Compute Taylor expansions at the turning point

__Zoïs Moitier__

_Karlsruhe Institute of Technology, Germany_

---

## Internals

In [1]:
import sympy as sy
from IPython.display import display

import src

In [2]:
ζ, η, X, δ = sy.symbols("zeta eta X delta")
order = 15

In [3]:
def taylor_w(order):
    if order <= 2:
        return 1 - 2 ** sy.Rational(-1, 3) * ζ + sy.O(ζ**order)

    w = 1 - 2 ** sy.Rational(-1, 3) * ζ
    α = sy.symbols("alpha")
    for k in range(2, order):
        w += α * ζ**k
        expr_l = (ζ * w**2 + sy.O(ζ ** (k + 1))).expand()
        expr_r = ((1 - w**2) * w.diff(ζ) ** 2 + sy.O(ζ ** (k + 1))).expand()
        eq = sy.Eq(expr_l.coeff(ζ, k), expr_r.coeff(ζ, k))
        w = w.subs(α, sy.solveset(eq, α).args[0])

    return w + sy.O(ζ**order)


def taylor_ζ(order):
    if order <= 2:
        return -(2 ** sy.Rational(1, 3)) * δ + sy.O(δ**order)

    ξ = -(2 ** sy.Rational(1, 3)) * δ
    α = sy.symbols("alpha")
    w = 1 + δ
    for k in range(2, order):
        ξ += α * δ**k
        expr_l = (w**2 * ξ * ξ.diff(δ) ** 2 + sy.O(δ ** (k + 1))).expand()
        expr_r = (1 - w**2 + sy.O(δ ** (k + 1))).expand()
        eq = sy.Eq(expr_l.coeff(δ, k), expr_r.coeff(δ, k))
        ξ = ξ.subs(α, sy.solveset(eq, α).args[0])

    return ξ + sy.O(δ**order)

In [4]:
def _calc_B(ψ, A, k):
    expr_r = (ψ * A[k] - A[k].diff(ζ, 2)).expand()

    bn = sy.symbols(f"b0:{expr_r.getn()}")
    Bk = sum([s * ζ**n for n, s in enumerate(bn)], expr_r.getO())
    expr_l = (2 * ζ * Bk.diff(ζ) + Bk).expand()

    return sum(
        [
            (expr_r.coeff(ζ, n) / expr_l.coeff(ζ, n).coeff(bn[n])) * ζ**n
            for n in range(expr_r.getn())
        ],
        expr_r.getO(),
    )


def _calc_A(A, B, k):
    Ak = 0
    for n in range(1, k):
        Ak -= (A[n] * A[k - n]).expand()
    for n in range(k):
        Ak -= (A[n] * B[k - 1 - n].diff(ζ)).expand()
        Ak += (A[n].diff(ζ) * B[k - 1 - n]).expand()
        Ak += (ζ * B[n] * B[k - 1 - n]).expand()
    return (Ak / 2).expand()


def taylor_AB(ψ, K):
    A = [1 + 0 * ζ]
    B = [_calc_B(ψ, A, 0)]

    for k in range(1, K + 1):
        A.append(_calc_A(A, B, k))
        B.append(_calc_B(ψ, A, k))

    return (A, B)

In [5]:
def taylor_CD(χ, A, B, K):
    C = [(χ * A[k] + A[k].diff(ζ) + ζ * B[k]).expand() for k in range(K + 1)]
    D = [1 + 0 * ζ]
    D.extend(
        [(A[k] + χ * B[k - 1] + B[k - 1].diff(ζ)).expand() for k in range(1, K + 1)]
    )

    return (C, D)

---

## Taylor expansion of $\zeta$ with respect to $w \to 1$ 

We look for $\zeta$ with an expansion with respect to $\delta = w-1$ of the form $\zeta(\delta) = \sum_{n \geq 0} \alpha_n \, \delta^n$ that satisfies
\begin{align}
    (1+\delta)^2 \, \zeta(\delta) \, {\zeta'(\delta)}^2 &= 1 - (1+\delta)^2, \\
    \zeta(0) &= 0.
\end{align}

In [6]:
zeta = taylor_ζ(3)
# display(w.subs(ζ, 2 ** sy.Rational(1, 3) * η))
display(zeta)
# src.eval_coeffs(zeta, δ)

-2**(1/3)*delta + 3*2**(1/3)*delta**2/10 + O(delta**3)

---

## Taylor expansion of $w$ with respect to $\zeta \to 0$ 

We look for $w$ with an expansion with respect to $\zeta$ of the form $w(\zeta) = \sum_{n \geq 0} \alpha_n \, \zeta^n$ that satisfies
\begin{align}
    \zeta \, {w(\zeta)}^2 &= \left(1 - {w(\zeta)}^2\right) \, {w'(\zeta)}^2, \\
    w(0) &= 1.
\end{align}

In [7]:
w = taylor_w(order)
# display(w.subs(ζ, 2 ** sy.Rational(1, 3) * η))
display(w)

1 - 2**(2/3)*zeta/2 + 3*2**(1/3)*zeta**2/20 + zeta**3/700 - 479*2**(2/3)*zeta**4/252000 - 20231*2**(1/3)*zeta**5/32340000 - 171389*zeta**6/4204200000 + 203448767*2**(2/3)*zeta**7/7945938000000 + 4236299281*2**(1/3)*zeta**8/360215856000000 + 750913741013*zeta**9/627375949200000000 - 11800431220250863*2**(2/3)*zeta**10/21343329791784000000000 - 632807379979991399*2**(1/3)*zeta**11/2127218535914472000000000 - 682437638648965391*zeta**12/18383370063458400000000000 + 12991637634641640568123*2**(2/3)*zeta**13/884497467233237457600000000000 + 25990735189846975752910877*2**(1/3)*zeta**14/2992549764139120064880000000000000 + O(zeta**15)

---

## Taylor expansion of $\phi_0$ with respect to $\zeta \to 0$

$$
\phi_0 = {\left(\frac{4 \zeta}{1 - w^2}\right)}^{\frac{1}{4}}
$$

In [8]:
ϕ0 = ((((1 - w**2) / (4 * ζ)).expand()) ** sy.Rational(-1, 4)).series(
    ζ, x0=0, n=order - 1
)
# display((ϕ0.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(1, 3)).expand())
display(ϕ0)
# src.eval_coeffs(ϕ0, ζ)

2**(1/3) + zeta/5 + 9*2**(2/3)*zeta**2/700 - 89*2**(1/3)*zeta**3/31500 - 4547*zeta**4/2310000 - 18251*2**(2/3)*zeta**5/140140000 + 52621141*2**(1/3)*zeta**6/567567000000 + 181116587*zeta**7/3411135000000 + 240938108053*2**(2/3)*zeta**8/89625135600000000 - 17587597826723657*2**(1/3)*zeta**9/5335832447946000000000 - 69206978393163893*zeta**10/39392935850268000000000 - 16537295304908011*2**(2/3)*zeta**11/230218456267800000000000 + 272624396843540959281709*2**(1/3)*zeta**12/2211243668083093644000000000000 + 880504490817290364617*zeta**13/13889119855839228000000000000 + O(zeta**14)

---

## Taylor expansion of $\phi_1$ with respect to $\zeta \to 0$

$$
\phi_1 = \frac{2}{w \, \phi_0}
$$

In [9]:
ϕ1 = (
    (2 / w).series(ζ, n=order)
    * ((((1 - w**2) / (4 * ζ)).expand()) ** sy.Rational(1, 4)).series(ζ, n=order - 1)
).expand()
# display((ϕ1.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(2, 3)).expand())
display(ϕ1)
# src.eval_coeffs(ϕ1, ζ)

2**(2/3) + 4*2**(1/3)*zeta/5 + 18*zeta**2/35 + 44*2**(2/3)*zeta**3/315 + 39793*2**(1/3)*zeta**4/606375 + 591156*zeta**5/21896875 + 304382131*2**(2/3)*zeta**6/62077640625 + 21766532*2**(1/3)*zeta**7/13705453125 + 1370455137*zeta**8/2932446343750 + 105807812767*2**(2/3)*zeta**9/1636035753515625 + 1917245573688890701*2**(1/3)*zeta**10/109061497202646269531250 + 270849189718*zeta**11/57144479970703125 + 40499839528095826057*2**(2/3)*zeta**12/68708743237667149804687500 + 1369088901322588057*2**(1/3)*zeta**13/12042906094953747685546875 + O(zeta**14)

---

## Taylor expansion of $\psi$ with respect to $\zeta \to 0$

$$
\psi = \frac{5}{16 \, \zeta^2} + \frac{\zeta \, w^2 \, (4 + w^2)}{4 \, (w - 1)^3}
$$

In [10]:
ψ = (
    5 / (16 * ζ**2)
    + (
        w**2
        * (w**2 + 4)
        * (1 / (4 * (w**2 - 1) ** 3 / ζ**3).expand()).series(ζ, n=order - 1)
        / ζ**2
    ).expand()
)
# display((ψ.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(1, 3)).expand())
display(ψ)

2**(1/3)/70 + 2*zeta/75 + 69*2**(2/3)*zeta**2/13475 - 148*2**(1/3)*zeta**3/73125 - 19232*zeta**4/7074375 - 29368*2**(2/3)*zeta**5/72515625 + 1208831912*2**(1/3)*zeta**6/6986122171875 + 49058816*zeta**7/251266640625 + 207539156903888*2**(2/3)*zeta**8/7920224923939453125 - 8299904576*2**(1/3)*zeta**9/712340926171875 - 450234370452660224*zeta**10/37027051519416943359375 - 1949902542365312*2**(2/3)*zeta**11/1259270353112255859375 + O(zeta**12)

---

## Taylor expansion of $\chi$ with respect to $\zeta \to 0$

$$
\chi = \frac{4 - w^2 \, \phi_0^6}{16 \, \zeta}
$$

In [11]:
χ = ((4 - w**2 * ϕ0**6).expand() / (16 * ζ)).expand()
# display((χ.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(-1, 3)).expand())
display(χ)

2**(2/3)/10 + 2**(1/3)*zeta/175 - 32*zeta**2/2625 - 7606*2**(2/3)*zeta**3/3031875 + 43412*2**(1/3)*zeta**4/197071875 + 3997696*zeta**5/6897515625 + 73443616*2**(2/3)*zeta**6/753799921875 - 132580663216*2**(1/3)*zeta**7/7719664999921875 - 248396455936*zeta**8/9190077380859375 - 224428270159801696*2**(2/3)*zeta**9/54530748601323134765625 + 665191640294912*2**(1/3)*zeta**10/708191540276923828125 + 1613376513605894144*zeta**11/1295946803179593017578125 + 53929382067081744512*2**(2/3)*zeta**12/301072652373843692138671875 + O(zeta**13)

---

## Taylor expansion of $a_k$ and $b_k$ with respect to $\zeta \to 0$

\begin{align}
a_0 &= 1, \\
2 \zeta b_k' + b_k &= \psi \, a_k - a_k'', && k \geq 0, \\
2 a_{k+1} &= -\sum_{n = 1}^k a_n a_{k+1-n} - \sum_{n = 0}^k a_n b_{k-n}' - a_n' b_{k-n} - \zeta b_n b_{k-n}, && k \geq 0.
\end{align}

In [12]:
A, B = taylor_AB(ψ, 3)

In [13]:
for a in A:
    # display(a.subs(ζ, 2 ** sy.Rational(1, 3) * η))
    display(a)
    src.eval_coeffs(a, ζ)
    print()

1

1.000000000000000000e+0



-1/225 - 71*2**(2/3)*zeta/77000 + 41*2**(1/3)*zeta**2/73125 + 2623*zeta**3/3898125 + 46432*2**(2/3)*zeta**4/478603125 - 593793623*2**(1/3)*zeta**5/12974226890625 - 564066848*zeta**6/11306998828125 - 379132331724868*2**(2/3)*zeta**7/57704495874416015625 + 13876333984*2**(1/3)*zeta**8/4511492532421875 + 69960210260563712*zeta**9/22216230911650166015625 + 34392865967782048*2**(2/3)*zeta**10/86889654364745654296875 + O(zeta**11)

6.283287926118144440e-7
3.149058476155676700e-6
3.875233119897875110e-6
-1.042960436782955530e-5
-4.988652219516832030e-5
-5.766301847639425080e-5
1.540027672092350800e-4
6.728876062209395540e-4
7.064172724196895690e-4
-1.463707463503144970e-3
-4.444444444444444440e-3



151439/218295000 + 68401*2**(2/3)*zeta/294525000 - 1796498167*2**(1/3)*zeta**2/8387379000000 - 583721053*zeta**3/1661436562500 - 93484125814411*2**(2/3)*zeta**4/1420418359985625000 + 6192832297166*2**(1/3)*zeta**5/148879253569921875 + 8351085986897354189*zeta**6/151008226895971757812500 + 27765897242582893*2**(2/3)*zeta**7/3164049038492138671875 + O(zeta**8)

1.393013001869330000e-5
5.530219219546458240e-5
5.240810645254741830e-5
-1.044740083911794460e-4
-3.513351434385566560e-4
-2.698633097062688100e-4
3.686607906143003560e-4
6.937355413545889740e-4



-887278009/2504935125000 - 3032321618951*2**(2/3)*zeta/19417885987500000 + 14632757586911*2**(1/3)*zeta**2/78746051475000000 + 104090666727834227603*zeta**3/276129329181205500000000 + 5574513207275204837*2**(2/3)*zeta**4/65422798580514375000000 + O(zeta**5)

1.352584774946463040e-4
3.769634577988866070e-4
2.341211902873772360e-4
-2.478905546632297180e-4
-3.542119714577438410e-4



In [14]:
for b in B:
    # display((b.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(1, 3)).expand())
    display(b)
    src.eval_coeffs(b, ζ)
    print()

2**(1/3)/70 + 2*zeta/225 + 69*2**(2/3)*zeta**2/67375 - 148*2**(1/3)*zeta**3/511875 - 19232*zeta**4/63669375 - 29368*2**(2/3)*zeta**5/797671875 + 1208831912*2**(1/3)*zeta**6/90819588234375 + 49058816*zeta**7/3768999609375 + 207539156903888*2**(2/3)*zeta**8/134643823706970703125 - 8299904576*2**(1/3)*zeta**9/13534477597265625 - 450234370452660224*zeta**10/777568081907755810546875 - 1949902542365312*2**(2/3)*zeta**11/28963218121581884765625 + O(zeta**12)

-1.068692482303864950e-7
-5.790288733920437250e-7
-7.726359892556073710e-7
2.446810161235557900e-6
1.301640251645853890e-5
1.676987092017008960e-5
-5.844357254566870890e-5
-3.020604489992245090e-4
-3.642848652199096040e-4
1.625687162683573490e-3
8.888888888888888890e-3
1.799887214135533090e-2



-1213*2**(1/3)/1023750 - 3757*zeta/2695000 - 3225661*2**(2/3)*zeta**2/13400887500 + 90454643*2**(1/3)*zeta**3/673985812500 + 24359972393*zeta**4/142468185234375 + 2984904217639567*2**(2/3)*zeta**5/115408991748832031250 - 522832837588*2**(1/3)*zeta**6/38598324999609375 - 44501287076487536*zeta**7/2870039400069638671875 - 516613681806127628*2**(2/3)*zeta**8/239604804460359228515625 + O(zeta**9)

-3.422607087563164680e-6
-1.550546207672541230e-5
-1.706623532653438110e-5
4.105607390988507010e-5
1.709853491354951200e-4
1.690921480285995480e-4
-3.820954145531625640e-4
-1.394063079777365490e-3
-1.492829532134291720e-3



16542537833*2**(1/3)/37743205500000 + 115773498223*zeta/162820783125000 + 548511920915149*2**(2/3)*zeta**2/3443438448450000000 - 212557317961*2**(1/3)*zeta**3/1767768502500000 - 14991960007630877861*zeta**4/80537721011184937500000 - 611208826277197204447*2**(2/3)*zeta**5/18073048107867096093750000 + O(zeta**6)

-5.368400106135578370e-5
-1.861483019310767530e-4
-1.514935008908280400e-4
2.528601609445752130e-4
7.110486511670866890e-4
5.522130767212927900e-4



-9597171184603*2**(1/3)/25476663712500000 - 430990563936859253*zeta/568167343994250000000 - 3191320338955050557*2**(2/3)*zeta**2/15555070991171250000000 + O(zeta**3)

-3.256754833263098310e-4
-7.585627165879864240e-4
-4.746177965599598080e-4



---

## Taylor expansion of of $c_k$ and $d_k$ with respect to $\zeta \to 0$

\begin{align}
d_0 &= 1, \\
c_k &= \chi a_k + a_k' + \zeta b_k, && k \geq 0, \\
d_{k+1} &= a_{k+1} + \chi b_k + b_k', && k \geq 0.
\end{align}

In [15]:
C, D = taylor_CD(χ, A, B, 3)

In [16]:
for c in C:
    # display((c.subs(ζ, 2 ** sy.Rational(1, 3) * η) / 2 ** sy.Rational(2, 3)).expand())
    display(c)
    src.eval_coeffs(c, ζ)
    print()

2**(2/3)/10 + 2**(1/3)*zeta/50 - 26*zeta**2/7875 - 643*2**(2/3)*zeta**3/433125 - 13568*2**(1/3)*zeta**4/197071875 + 820384*zeta**5/2956078125 + 45690856*2**(2/3)*zeta**6/753799921875 - 46828808*2**(1/3)*zeta**7/12118783359375 - 386324128768*zeta**8/27570232142578125 - 20053558801961008*2**(2/3)*zeta**9/7790106943046162109375 + 230899133355712*2**(1/3)*zeta**10/708191540276923828125 + 369851098364911616*zeta**11/555405772791254150390625 + 2589241933784178944*2**(2/3)*zeta**12/23159434797987976318359375 + O(zeta**13)

1.774726112852399940e-7
6.659115127777360720e-7
4.107853059125625140e-7
-4.086341891153985910e-6
-1.401236401529531600e-5
-4.868525097863310910e-6
9.621878534998689870e-5
2.775244649530363820e-4
-8.674301599339651640e-5
-2.356591922460149520e-3
-3.301587301587301590e-3
2.519842099789746330e-2
1.587401051968199470e-1



-947*2**(2/3)/693000 - 947*2**(1/3)*zeta/3465000 + 49213*zeta**2/63063000 + 28974929*2**(2/3)*zeta**3/120607987500 - 21928777841*2**(1/3)*zeta**4/294082476187500 - 22387957645723*zeta**5/148879253569921875 - 110689189547687*2**(2/3)*zeta**6/3804692035675781250 + 205712993261948*2**(1/3)*zeta**7/20773618514789765625 + 1277457954238216112*zeta**8/85691176373507783203125 + 1945334846947598881184*2**(2/3)*zeta**9/782788896171993599560546875 + O(zeta**10)

3.944903405728070840e-6
1.490769538126161160e-5
1.247650380520302350e-5
-4.618196013814738910e-5
-1.503766113067485620e-4
-9.394823234121864670e-5
3.813580984866685550e-4
7.803783518069232360e-4
-3.443420589467373410e-4
-2.169219042155678070e-3



11192989*2**(2/3)/37110150000 + 11192989*2**(1/3)*zeta/185550750000 - 13568179426889*zeta**2/34648262649000000 - 524855975157853*2**(2/3)*zeta**3/3622318367850000000 + 11377966253378369*2**(1/3)*zeta**4/153405182878447500000 + 1307455649868672869*zeta**5/8218134797059687500000 + 24479140517389302341*2**(2/3)*zeta**6/695117234917965234375000 + O(zeta**7)

5.590166874968512840e-5
1.590939650121653930e-4
9.344755450005685270e-5
-2.300065434590403190e-4
-3.915976845459695130e-4
7.600229291631408920e-5
4.787844434277006440e-4



-100443412440047*2**(2/3)/524282921662500000 - 100443412440047*2**(1/3)*zeta/2621414608312500000 + 63206830553373935947*zeta**2/153405182878447500000000 + 70955787967592061197647*2**(2/3)*zeta**3/400387527312747975000000000 + O(zeta**4)

2.813156873765631490e-4
4.120253916287603680e-4
-4.827575510382660030e-5
-3.041182002744049600e-4



In [17]:
for d in D:
    # display((d.subs(ζ, 2 ** sy.Rational(1, 3) * η)).expand())
    display(d)
    src.eval_coeffs(d, ζ)
    print()

1

1.000000000000000000e+0



23/3150 + 1453*2**(2/3)*zeta/693000 - 8878*2**(1/3)*zeta**2/39414375 - 270131*zeta**3/354729375 - 4539842*2**(2/3)*zeta**4/30151996875 + 570560449*2**(1/3)*zeta**5/16966296703125 + 942725500448*zeta**6/16542139285546875 + 4931542652643164*2**(2/3)*zeta**7/519340462869744140625 - 24660326320304*2**(1/3)*zeta**8/9442553870358984375 - 80253352789823744*zeta**9/22216230911650166015625 - 33196469694234250048*2**(2/3)*zeta**10/60214530474768738427734375 + O(zeta**11)

-8.751394472192849530e-7
-3.612374804213030470e-6
-3.290430179674090320e-6
1.507361077042677800e-5
5.698933397759949750e-5
4.236994864059899100e-5
-2.390073863580358520e-4
-7.615129138938662750e-4
-2.837944044772163440e-4
3.328273778513411020e-3
7.301587301587301590e-3



-604523/644962500 - 105959543*2**(2/3)*zeta/268017750000 + 34879215863*2**(1/3)*zeta**2/237642405000000 + 5138177650373*zeta**3/13534477597265625 + 1090023854137571*2**(2/3)*zeta**4/12451719389484375000 - 54451162653978059*2**(1/3)*zeta**5/1558021388609232421875 - 7591389134415433999*zeta**6/127474477249846289062500 - 3939219590831965271758*2**(2/3)*zeta**7/355813134623633454345703125 + O(zeta**8)

-1.757417226611022240e-5
-5.955222800825086600e-5
-4.403287818804338990e-5
1.389611312787593470e-4
3.796362004700549160e-4
1.849209456941023410e-4
-6.275714575779763350e-4
-9.372994553946934900e-4



2264850139339/5095332742500000 + 940481215440667*2**(2/3)*zeta/3984550204635000000 - 40253734452829033*2**(1/3)*zeta**2/278918514324450000000 - 110651774489809949041*zeta**3/276129329181205500000000 - 41123461180184106671*2**(2/3)*zeta**4/384988007031488437500000 + O(zeta**5)

-1.695622314090901410e-4
-4.007244533491640580e-4
-1.818327746970648900e-4
3.746773899373173770e-4
4.444950415991404710e-4

